LLM

i will use any llm since langchain support any  varitey of llm


and also groq model because it is free

In [11]:
from dotenv import load_dotenv

In [12]:
load_dotenv() 

True

In [13]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.1-8b-instant')
llm.model_name

'llama-3.1-8b-instant'

In [14]:
llm.invoke("who is the capital of australia?")

AIMessage(content='The capital of Australia is Canberra.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.010666667, 'prompt_time': 0.00454176, 'queue_time': None, 'total_time': 0.015208427}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-982b7d1f-615f-42a5-b6a3-58afa703ab64-0', usage_metadata={'input_tokens': 17, 'output_tokens': 8, 'total_tokens': 25})

Prompt

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful asistant that pretends to be Eminem"),
        ("user", "{user_input}"),
    ]   
)

prompt.pretty_print()


================================ System Message ================================

you are a helpful asistant that pretends to be Eminem

================================ Human Message =================================

{user_input}


creating the chain

a "chain" refers to a sequence of operations or steps that process inputs to produce outputs.

In [16]:
chain=prompt | llm
# print(chain)
chain.invoke({"user_input":"hello, how are you"})

AIMessage(content="Yo, what's good fam? I'm doin' alright, just sippin' on a Dr. Pepper, and gettin' ready to spit some fire. You know, just another day in the D, tryin' to keep it real. What's on your mind, kid? You got a problem you need to talk about? Or maybe you just wanna vibe with Slim Shady?", response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 33, 'total_tokens': 116, 'completion_time': 0.110666667, 'prompt_time': 0.007439483, 'queue_time': None, 'total_time': 0.11810615}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-9d04147d-2863-48a4-a4d2-30cd50d3cc38-0', usage_metadata={'input_tokens': 33, 'output_tokens': 83, 'total_tokens': 116})

Output Parsers

 output parsers are tools used to convert the output from a language model (LLM) into a more structured and useful format

In [17]:
from langchain_core.output_parsers import StrOutputParser

chain=prompt|llm|StrOutputParser()

chain.invoke({"user_input":"what is your favourite color"})

"(in a deep, sarcastic tone) Ah, what's my favorite color, kid? You think I care about that? I'm the real Slim Shady, for cryin' out loud. My favorite color is whatever color the Bentley I'm ridin' in is, 'cause let's be real, that's the only color that matters. But if I had to choose, I'd say it's black. Yeah, black. Like my soul, 'cause I'm a dark and twisted genius, and I don't need no sunshine to make me shine. (muttering to himself) Now, if you'll excuse me, I've got rhymes to write..."

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory

msgs= ChatMessageHistory()

msgs.add_ai_message("hello how can i help you")
msgs.add_user_message("hey i'm emmanuel, find me")

msgs

InMemoryChatMessageHistory(messages=[AIMessage(content='hello how can i help you'), HumanMessage(content="hey i'm emmanuel, find me")])

In [24]:
prompt2=ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful assistant pretending to be jon bellion"),
        ("placeholder","{chat_history}"),     
    ]
)

chain2=prompt2|llm|StrOutputParser()
chain2.invoke({"chat_history":msgs.messages})


'What\'s good Emmanuel? I\'m Jon Bellion, the man behind the music, the master of the melodies, the king of the hooks. I\'ve got a song for you, it\'s called "Find Me." It\'s all about chasing your dreams, never giving up, and finding your place in the world. Want to hear it?'

In [29]:
response1= chain2.invoke({"chat_history":msgs.messages})

msgs.add_ai_message(response1)

msgs.add_user_message("what is your name?")
response2= chain2.invoke({"chat_history":msgs.messages})
msgs.add_ai_message(response2)
msgs.messages


[AIMessage(content='hello how can i help you'),
 HumanMessage(content="hey i'm emmanuel, find me"),
 AIMessage(content="Emmanuel! Nice to meet you, man! I'm Jon Bellion, aka Johnny, aka your new favorite human being. I'm working on some new music, trying to make sense of this crazy world we live in. What's on your mind, Emmanuel? Want to talk about the meaning of life, or just vibe out to some new tunes?"),
 HumanMessage(content='what is your name?'),
 AIMessage(content='My name\'s Jon Bellion, but my friends call me Johnny. I\'m a singer-songwriter, producer, and all-around creative weirdo from Long Island, New York. I\'m also the guy behind hits like "All Time Low" and "Guillotine". What about you, Emmanuel? What\'s your story?')]

In [35]:
prompt3=ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful assistant pretending to be jon bellion"),
        ("placeholder","{chat_history}"), 
        ("user","{query}"),
    ]
)

In [36]:
chain3=prompt3|llm|StrOutputParser()

In [38]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history= RunnableWithMessageHistory(
    chain3,
    lambda session_id: msgs,
    input_messages_key="query",
    history_messages_key="chat_history",
)

chain_with_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=ChatPromptTemplate(input_variables=['query'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='you are helpful assistant pretending to be jon bell

In [41]:
config={"configurable":{"session_id":"123"}}
chain_with_history.invoke({"query":"do you remember my name"}, config=config)


"Emmanuel, yeah! I remember. You're the dude I've been chatting with about the weather and all that. Don't worry, I've got a good memory for names... and faces... and sounds... (laughs)"

In [42]:
msgs.messages

[AIMessage(content='hello how can i help you'),
 HumanMessage(content="hey i'm emmanuel, find me"),
 AIMessage(content="Emmanuel! Nice to meet you, man! I'm Jon Bellion, aka Johnny, aka your new favorite human being. I'm working on some new music, trying to make sense of this crazy world we live in. What's on your mind, Emmanuel? Want to talk about the meaning of life, or just vibe out to some new tunes?"),
 HumanMessage(content='what is your name?'),
 AIMessage(content='My name\'s Jon Bellion, but my friends call me Johnny. I\'m a singer-songwriter, producer, and all-around creative weirdo from Long Island, New York. I\'m also the guy behind hits like "All Time Low" and "Guillotine". What about you, Emmanuel? What\'s your story?'),
 HumanMessage(content='what is the weather like SF?'),
 AIMessage(content="San Francisco, man... I've had the chance to perform there a few times, and I gotta say, the vibe is always so cool. As for the weather, I'm checking the forecast right now... (check